In [107]:
pip install easy_vqa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [108]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import transforms
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

class VQAModel(nn.Module):
    def __init__(self, num_answers):
        super(VQAModel, self).__init__()
        
        # image feature extraction with pre-trained ResNet-18 model
        self.resnet = models.resnet18(pretrained=True).to(device)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1]).to(device) # remove last fully connected layer
        
        # quesetion feature extraction using bag of words (BoW) model
        self.bow = CountVectorizer()
        
        # combine image and question features using a fully connected layer
        self.fc = nn.Sequential(
            nn.Linear(512 + 512, 256),
            nn.ReLU(),
            nn.Linear(256, num_answers)
        ).to(device)

    def forward(self, image, question):
        # Extract image features
        image_features = self.resnet(image)
        image_features = image_features.view(image_features.size(0), -1)

        # combine image & question features, pad with 0s
        extra = torch.zeros((len(question), 512 - len(question[0]))).to(device)
        combined_features = torch.cat((image_features, question), dim=1).to(device)
        combined_features = torch.cat((combined_features, extra), dim = 1).to(device)

        # output logits for each answer choice
        return self.fc(combined_features)

Using device: cuda


In [102]:
from easy_vqa import get_train_questions, get_train_image_paths, get_test_questions, get_test_image_paths, get_answers
from PIL import Image

def load_dataset():
    # load questions, image paths, and answers
    train_qs, train_answers, train_image_ids = get_train_questions()
    test_qs, test_answers, test_image_ids = get_test_questions()
    train_image_paths = get_train_image_paths()
    test_image_paths = get_test_image_paths()
    answers = get_answers()

    # preprocess the images
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_images = [transform(Image.open(train_image_paths[id]).convert('RGB')) for id in train_image_ids[:200]]
    test_images = [transform(Image.open(test_image_paths[id]).convert('RGB')) for id in test_image_ids[:200]]

    return train_images[:200], train_qs[:200], train_answers[:200], test_images[:200], test_qs[:200], test_answers[:200], answers

In [103]:
from torch.utils.data import Dataset, DataLoader

class VQADataset(Dataset):
    def __init__(self, images, questions, answers):
        self.images = images
        self.questions = questions
        
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.images[idx], self.questions[idx], self.answers[idx]

def create_dataloader(images, questions, answers, batch_size=64):
    dataset = CustomDataset(images, questions, answers)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

class CustomDataset(Dataset):
    def __init__(self, images, questions, answers):
        self.images = images
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.images[idx], self.questions[idx], self.answers[idx]

In [104]:
import torch.optim as optim

def train_vqa_model(model, train_dataloader, epochs=10, learning_rate=0.001):
    model.train() # set model to training mode
    # loss function & optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        running_loss = 0.0
        for images, questions, answers in train_dataloader:

            images = images.to(device)
            # fit_transform
            questions = model.bow.transform([q for q in questions])
            questions = torch.tensor(questions.todense()).float().to(device)

            # convert answers to numerical labels
            int_labels = []
            for ans in answers:
                int_labels.append(label_dict[ans])
            int_labels = torch.tensor(int_labels).to(device)

            optimizer.zero_grad()
            outputs = model(images, questions)
            loss = criterion(outputs, int_labels).to(device)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_dataloader)}')


train_images, train_qs, train_answers, test_images, test_qs, test_answers, answers = load_dataset()

label_dict = {}
i = 0
# assign index to each of 13 possible answers
for ans in answers:
    label_dict[ans] = i
    i += 1

# initialize the VQA model with 13 answer choices (Easy-VQA dataset)
model = VQAModel(num_answers=13)

# fit the BoW model with the vocabulary of the questions in the dataset
model.bow.fit(train_qs)

train_dataloader = create_dataloader(train_images, train_qs, train_answers)
train_vqa_model(model, train_dataloader, 100)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100, Loss: 2.0141628682613373
Epoch 2/100, Loss: 1.724893033504486
Epoch 3/100, Loss: 1.5875495076179504
Epoch 4/100, Loss: 1.4964751601219177
Epoch 5/100, Loss: 1.4175163507461548
Epoch 6/100, Loss: 1.3211841881275177
Epoch 7/100, Loss: 1.3830206990242004
Epoch 8/100, Loss: 1.414231687784195
Epoch 9/100, Loss: 1.361294448375702
Epoch 10/100, Loss: 1.334049940109253
Epoch 11/100, Loss: 1.2235138416290283
Epoch 12/100, Loss: 1.2607295215129852
Epoch 13/100, Loss: 1.2078235447406769
Epoch 14/100, Loss: 1.1810083985328674
Epoch 15/100, Loss: 1.2362671196460724
Epoch 16/100, Loss: 1.2857175767421722
Epoch 17/100, Loss: 1.1699181497097015
Epoch 18/100, Loss: 1.2079658806324005
Epoch 19/100, Loss: 1.3222905099391937
Epoch 20/100, Loss: 1.2707453966140747
Epoch 21/100, Loss: 1.231934279203415
Epoch 22/100, Loss: 1.3081634044647217
Epoch 23/100, Loss: 1.18614062666893
Epoch 24/100, Loss: 1.288623183965683
Epoch 25/100, Loss: 1.24125736951828
Epoch 26/100, Loss: 1.149610549211502
Epoch 

In [105]:
def test_vqa_model(model, test_images, test_questions, test_answers):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(len(test_images)):
            image = test_images[i].unsqueeze(0).to(device)
            question = model.bow.transform([test_questions[i]])
            question = torch.tensor(question.todense()).float().to(device)
            
            output = model(image, question)
            _, predicted = torch.max(output, 1)
            total += 1
            # print(test_answers[i], label_dict[test_answers[i]])
            # print(predicted)
            correct += sum(predicted == label_dict[test_answers[i]]).item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100}%')
    return accuracy

In [106]:
train_images, train_qs, train_answers, test_images, test_qs, test_answers, answers = load_dataset()
test_dataloader = create_dataloader(test_images, test_qs, test_answers)
test_vqa_model(model, test_images, test_qs, test_answers)

Accuracy: 59.0%


0.59